In [1]:
import os
from pathlib import Path

os.chdir("..")


In [ ]:
from open_extract.preprocess import pdf2jpg
from open_extract.data_model import Article

In [8]:
for x in Path("data/pdfs").glob("*.pdf"):
    pdf2jpg(x, Path("data/images") / x.stem)


In [ ]:
list(map(str, Path("data/images/1").glob("*.jpg")))

In [ ]:
import ollama
from tqdm import tqdm

client = ollama.Client("http://olvi-1:11434")

for i in tqdm(list(Path("data/images").rglob("*.jpg"))):
    try:
        messages = [
            {
                "role": "user",
                "content": "Extract data all the images",
                "images": [str(i)],
            }
        ]

        response = client.chat(model="custom-vision-model", messages=messages, format=Article.model_json_schema())
        a = Article.model_validate_json(response.message.content)
        a.save(i.with_suffix(".json"))
    except Exception as e:
        print(e)
        pass


tl;dr

1. llama3.2 vision only works on 1 page, that means we need to adopt agentic workflow and segment the fig and table... 
2. Somewhat low res image at the current config
3. Somewhat compute hungry